    /# Structure of the files after modularization  
https://www.learnpytorch.io/05_pytorch_going_modular/

- `data_setup.py` 
> A file to preprare and download ata if needed.

- `engine.py`
> A file containing various training functions

- `model_builder.py` or `model.py`  
> A file to create a PyTorch model

- `train.py`
> A file to leverage all other files and train a target PyTorch model

- `utils.py`
> A file dedicated to helpful utility functions.


### Better strategy
- First Write your code in Jupyter Notebook for experiments and test.

- Transfering your code from the Jupyter Notebook to Modularized python files. 

---

# Example of actual production
```shell
python train.py --model MODEL_NAME --batch_size BATCH_SIZE --lr LEARNING_RATE --num_epochs NUM_EPOCHS

```

![example of actual production terminal command](../imgs/05-python-train-command-line-annotated.png)
![](https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/05-training-sota-recipe.png)

```shell
going_modular/
├── going_modular/
│   ├── data_setup.py
│   ├── engine.py
│   ├── model_builder.py
│   ├── train.py
│   └── utils.py
├── models/
│   ├── 05_going_modular_cell_mode_tinyvgg_model.pth
│   └── 05_going_modular_script_mode_tinyvgg_model.pth
└── data/
    └── pizza_steak_sushi/
        ├── train/
        │   ├── pizza/
        │   │   ├── image01.jpeg
        │   │   └── ...
        │   ├── steak/
        │   └── sushi/
        └── test/
            ├── pizza/
            ├── steak/
            └── sushi/
```

In [1]:
%%writefile test.txt
`This following text is a test for %%writefile file.txt.`

Overwriting test.txt


# Get data

In [15]:
import os
import requests
import zipfile
from pathlib import Path

# Create a folder for the modularized python files
folder_path = './modularization'

if os.path.isdir(folder_path):
    print(f"The folder at {folder_path} exists.")
else:
    print(f"The folder at {folder_path} does not exist.")
    print('\nCreating one...')
    os.mkdir('./modularization')

# Setup path to data folder
data_path = Path('../datasets/food101')
image_path = data_path / 'pizza_steak_sushi'

# print(image_path)

# If the image folder doesn't exist, download and create it.
if image_path.is_dir():
    print(f'{image_path} directory exists.')
    
else:
    print(f'Did not find {image_path} directory, creating one...')
    image_path.mkdir(parents=True, exist_ok=True)
    
    # Download the files -> pizza, steak, sushi
    with open(data_path / 'pizza_steak_sushi.zip', 'wb') as f:
        request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
        print('Downloading pizza, steak, sushi data...')
        f.write(request.content)   # write the data to the zip file

    # Unzip the file 
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:   
        print('Unzipping pizza, steak, ...')
        zip_ref.extractall(image_path) # extract zip to the image_path folder

The folder at ./modularization exists.
../datasets/food101/pizza_steak_sushi directory exists.


# Create Datasets and DataLoaders(data_setup.py)

In [16]:
%%writefile modularization/test.py

print('Hello world!')

Writing modularization/test.py


In [35]:
%%writefile modularization/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for 
image classification data.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
    """
    Creates training and testing DataLoaders.

    Takes in a training directory and testing directory path and turns
    them into PyTorch Datasets and then into PyTorch DataLoaders.

    Args:
        train_dir: Path to training directory.
        test_dir: Path to testing directory.
        transform: torchvision transforms to perform on training and testing data.
        batch_size: Number of samples per batch in each of the DataLoaders.
        num_workers: An integer for number of workers per DataLoader.

    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
        Example usage:
            train_dataloader, test_dataloader, class_names = \
                = create_dataloaders(train_dir=path/to/train_dir,
                                 test_dir=path/to/test_dir,
                                 transform=some_transform,
                                 batch_size=32,
                                 num_workers=4)

    """
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)
    
    
    # Get class names
    class_names = train_data.classes
    
    # Turn images into data loaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,   # CPU <-> GPU faster
    )
    
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    
    
    return train_dataloader, test_dataloader, class_names

Overwriting modularization/data_setup.py


In [39]:
%%writefile modularization/model_builder.py
"""
Pytorch model code to instantiate a TinyVGG model
"""

import torch
from torch import nn

class TinyVGG(nn.Module):
    """
    Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
        input_shape: An integer indicating number of input channels.
        hidden_units: An integer indicating number of hidden units between layers.
        output_shape: An integer indicating number of output units.
    """
    
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(
                in_channels=input_shape,
                out_channels=hidden_units,
                kernel_size=3,
                stride=1,
                padding=0
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                stride=1,
                padding=0
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*13*13,
                      out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x
        # return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Overwriting modularization/model_builder.py


# Instead of coding the TinyVGG, now we are able to import it 

```python
import torch
# Import model_builder.py
from modularization import model_builder
device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate an instance of the model from the "model_builder.py" script
torch.manual_seed(42)
model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=10, 
                              output_shape=len(class_names)).to(device)
```

# Creating `train_step()`, `test_step`, and `train()`

In [25]:
%%writefile modularization/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""

from typing import Tuple, Dict, List
import torch
from tqdm.auto import tqdm


def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """
    Trains a PyTorch model for a single epoch.
    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
        model: A PyTorch model to be trained.
        dataloader: A DataLoader instance for the model to be trained on.
        loss_fn: A PyTorch loss function to minimize.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
        A tuple of training loss and training accuracy metrics.
        In the form (train_loss, train_accuracy). For example:
        (0.1112, 0.8743)
    """
    
    # Put the model in training mode
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0
    
    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send ata to target devcie
        X, y = X.to(device), y.to(device)
        
        # 1. Forward pass
        y_pred = model(X)
        
        # 2. Calcualte and accumulate loss across batches
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        
        # 3. Optimizer zero grad
        """
        During backpropagation, gradients are accumulated in the 
        `.grad` attribute of each tensor that requires gradients.
        If `optimizer.zero_grad()` is not called before the next 
        iteration, these gradients will be accumulated on top of 
        the previous ones, which will lead to incorrect parameter 
        updates and suboptimal performance.
        """
        optimizer.zero_grad()
        
        # 4. Loss backward  -> backpropagation
        loss.backward()
        
        # 5. Optimizer step -> update weight, params
        optimizer.step()
        
        # Calculate and accumulate acc metric across batches
        y_pred_class = torch.argmax(torch.softmax(y_pred,
                                                  dim=1),
                                    dim=1)
        # Extract the correct predictions, compute the percentage of 
        # the correct predictions over the current batche
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)
        
    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """
    Tests a PyTorch model for a single epoch.
    
    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.
    
    Args:
        model: A PyTorch model to be tested.
        dataloader: A DataLoader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to calculate loss on the test data.
        device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
        A tuple of testing loss and testing accuracy metrics.
        In the form (test_loss, test_accuracy). For example:
        (0.0223, 0.8985)
  """
    # Put model in eval mode
    model.eval() 

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0
    
    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        
    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc


def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int, 
          device: torch.device) -> Dict[str, List]:
    """
    Trains and tests a PyTorch model
    
    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.
    
    Args:
        model: A PyTorch model to be trained and tested.
        train_dataloader: A DataLoader instance for the model to be trained on.
        test_dataloader: A DataLoader instance for the model to be tested on.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        loss_fn: A PyTorch loss function to calculate loss on both datasets.
        epochs: An integer indicating how many epochs to train for.
        device: A target device to compute on (e.g. "cuda" or "cpu").
    
    Returns:
        A dictionary of training and testing loss as well as training and
        testing accuracy metrics. Each metric has a value in a list for 
        each epoch.
        In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]} 
        For example if training for epochs=2: 
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]} 
    
    """
    
    # Create empty results dictionary
    results = {'train_loss': [],
               'train_acc': [],
               'test_loss': [],
               'test_acc': []
              }
    
    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)
        
        # Print out what's happening
        print(
            f'Epoch: {epoch + 1} | '
            f'train_loss: {train_loss: .4f} | '
            f'train_acc: {train_loss: .4f} | '
            f'test_loss: {test_loss: .4f} | '
            f'test_acc: {test_loss: .4f}'
        )
        
        # Update results dictionary
        results['train_loss'].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
    # Return the filled results at the end of the epochs
    return results

Overwriting modularization/engine.py


### Demo
```python
# Import engine.py
from modularization import engine

# Use train() by calling it from engine.py
engine.train(...)
```

# Creating a function to save the model(utils.py)

In [27]:
%%writefile modularization/utils.py
"""
Contains various utility functions for PyTorch model 
training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """
    Saves a PyTorch model to a target directory.
    
    Args:
        model: A target PyTorch model to save.
        target_dir: A directory for saving the model to.
        model_name: A filename for the saved model. Should include
        either ".pth" or ".pt" as the file extension.

    Example usage:
        save_model(model=model_0,
        target_dir="models",
        model_name="05_going_modular_tingvgg_model.pth")
    """
    
    # Create target dictionary
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                          exist_ok=True)
    
    # Create model save path
    assert model_name.endswith('.pth') or model_name.endswith('.pt'), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name
    
    # Save the model state_dict()
    print(f'[INFO] Saving model to: {model_save_path}')
    torch.save(obj=model.state_dict(),
               f=model_save_path)

Overwriting modularization/utils.py


### Demo
```python

# Import utils.py
from modularization import utils

# Save a model to file
save_model(model=...
           target_dir=...,
           model_name=...)
```

# Train, evaluate and save the model(train.py)

```shell
python train.py
```  

1.  Import the various dependencies, namely `torch`, `os`, `torchvision.transforms` and all of the scripts from the `going_modular` directory, `data_setup`, `engine`, `model_builder`, `utils`.
2.  **Note:** Since `train.py` will be _inside_ the `going_modular` directory, we can import the other modules via `import ...` rather than `from going_modular import ...`.
3.  Setup various hyperparameters such as batch size, number of epochs, learning rate and number of hidden units (these could be set in the future via [Python's `argparse`](https://docs.python.org/3/library/argparse.html)).
4.  Setup the training and test directories.
5.  Setup device-agnostic code.
6.  Create the necessary data transforms.
7.  Create the DataLoaders using `data_setup.py`.
8.  Create the model using `model_builder.py`.
9.  Setup the loss function and optimizer.
10.  Train the model using `engine.py`.
11.  Save the model using `utils.py`.

In [34]:
%%writefile modularization/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = .001

# Setup directories
train_dir = '../../datasets/food101/pizza_steak_sushi/train/'
test_dir = '../../datasets/food101/pizza_steak_sushi/test/'

# Setup target device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

##################### Create transforms ##########################
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])


############## Create DataLoader with help from data_setup.py ###########################################
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

################# Create model with help from model_builder.py#######
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

############# Set loss and optimizer ##########################
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir='./models',
                 model_name='jupyter_to_modular_tinyvgg.pth')

Overwriting modularization/train.py


# Output in terminal
![](./modularization/output.png)

In [ ]:
import argparse

parser = argparse.ArgumentParser(description='A script that accepts multiple flags')

parser.add_argument('-a', '--flag-a', action='store_true',
                    help='a flag that does something')
parser.add_argument('-b', '--flag-b', action='store_true',
                    help='another flag that does something')

args = parser.parse_args()

if args.flag_a:
    print('Flag A was set!')
else:
    print('Flag A was not set.')

if args.flag_b:
    print('Flag B was set!')
else:
    print('Flag B was not set.')
    
# python train.py --model MODEL_NAME --batch_size BATCH_SIZE --lr LEARNING_RATE --num_epochs NUM_EPOCHS

In [ ]:
# %%writefile modularization/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = .001

# Setup directories
train_dir = '../../datasets/food101/pizza_steak_sushi/train/'
test_dir = '../../datasets/food101/pizza_steak_sushi/test/'

# Setup target device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

##################### Create transforms ##########################
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])


############## Create DataLoader with help from data_setup.py ###########################################
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

################# Create model with help from model_builder.py#######
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

############# Set loss and optimizer ##########################
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir='./models',
                 model_name='jupyter_to_modular_tinyvgg.pth')